In [3]:
import numpy as np
from ortools.sat.python import cp_model

In [4]:
puzzle_size = 5

In [174]:
model = cp_model.CpModel()

In [175]:
start_numbers = {
    5: 4,
    9: 2,
    12: 4,
    19: 4
}

start_numbers.keys()

dict_keys([5, 9, 12, 19])

In [176]:
grid = np.array([model.NewIntVar(start_numbers.get(i, 1), 
                                 start_numbers.get(i, 5), 
                                 f'{i}') for i in range(puzzle_size ** 2)])

grid = grid.reshape((puzzle_size, -1))

In [177]:
grid

array([[0(1..5), 1(1..5), 2(1..5), 3(1..5), 4(1..5)],
       [5(4), 6(1..5), 7(1..5), 8(1..5), 9(2)],
       [10(1..5), 11(1..5), 12(4), 13(1..5), 14(1..5)],
       [15(1..5), 16(1..5), 17(1..5), 18(1..5), 19(4)],
       [20(1..5), 21(1..5), 22(1..5), 23(1..5), 24(1..5)]], dtype=object)

In [178]:
grid.shape

(5, 5)

In [179]:
# need to add constraint to all rows and column -- all different
for row in grid:
    print(row)
    model.AddAllDifferent(row)

[0(1..5) 1(1..5) 2(1..5) 3(1..5) 4(1..5)]
[5(4) 6(1..5) 7(1..5) 8(1..5) 9(2)]
[10(1..5) 11(1..5) 12(4) 13(1..5) 14(1..5)]
[15(1..5) 16(1..5) 17(1..5) 18(1..5) 19(4)]
[20(1..5) 21(1..5) 22(1..5) 23(1..5) 24(1..5)]


In [180]:
for column in grid.T:
    print(column)
    model.AddAllDifferent(column)
    

[0(1..5) 5(4) 10(1..5) 15(1..5) 20(1..5)]
[1(1..5) 6(1..5) 11(1..5) 16(1..5) 21(1..5)]
[2(1..5) 7(1..5) 12(4) 17(1..5) 22(1..5)]
[3(1..5) 8(1..5) 13(1..5) 18(1..5) 23(1..5)]
[4(1..5) 9(2) 14(1..5) 19(4) 24(1..5)]


In [181]:
grid[0,1]

1(1..5)

Create inequality constraints  
data format as tuples, 
* `[0]` = is on the lower side
* `[1]` = is on the higher side

In [182]:
ineqs = [(1,0), (3,2), (4,3), (18,19), (20,21), (21,22)]

In [183]:
ineqs = [
    ((0,1), (0,0)),
    ((0,3), (0,2)),
    ((0,4), (0,3)),
    ((3,3), (3,4)),
    ((4,0), (4,1)),
    ((4,1), (4,2))
    ]

In [184]:
grid[ineqs[1][1]]

2(1..5)

In [185]:
for i in ineqs:
    model.Add(grid[i[0]] < grid[i[1]])

In [186]:
solver = cp_model.CpSolver()

In [187]:
for row in grid:
    print(row.tolist())

[0(1..5), 1(1..5), 2(1..5), 3(1..5), 4(1..5)]
[5(4), 6(1..5), 7(1..5), 8(1..5), 9(2)]
[10(1..5), 11(1..5), 12(4), 13(1..5), 14(1..5)]
[15(1..5), 16(1..5), 17(1..5), 18(1..5), 19(4)]
[20(1..5), 21(1..5), 22(1..5), 23(1..5), 24(1..5)]


In [188]:
class DiagramPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables, ineqs):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__ineqs = ineqs
    
    def OnSolutionCallback(self):
        self.__solution_count += 1
 
        for row in self.__variables:
            row = row.tolist()
            vals = [self.Value(val) for val in row]
            print("+---+---+---+---+---+")
            print('| ', end = '')
            print(' | '.join(str(x) for x in vals), end = '')
            print(' |')
        
        print("+---+---+---+---+---+")
    
    def SolutionCount(self):
        return self.__solution_count

In [189]:
solution_printer = DiagramPrinter(grid, ineqs)

In [190]:
status = solver.SearchForAllSolutions(model, solution_printer)

+---+---+---+---+---+
| 5 | 4 | 3 | 2 | 1 |
+---+---+---+---+---+
| 4 | 3 | 1 | 5 | 2 |
+---+---+---+---+---+
| 2 | 1 | 4 | 3 | 5 |
+---+---+---+---+---+
| 3 | 5 | 2 | 1 | 4 |
+---+---+---+---+---+
| 1 | 2 | 5 | 4 | 3 |
+---+---+---+---+---+


In [191]:
print('Solutions found : %i' % solution_printer.SolutionCount())

Solutions found : 1


In [324]:
print(u'\u22C0')
print(u'\u22C1')
print('<')
print('>')

⋀
⋁
<
>


In [257]:
from tkinter import Entry
from tkinter import Tk
from tkinter import Button
from tkinter import END

In [312]:
class SampleApp(tk.Tk):
    
    def __init__(self):
        tk.Tk.__init__(self)
        self.title('Futoshiki Puzzle Setup')
        self.entry_grid(grid)
        self.button = Button(self, text="Submit", command=self.on_button, width = 6)
        self.button.grid(row = 5, columnspan = 5)
        
    def entry_grid(self, grid):
        self.entries = []
        for index, _ in np.ndenumerate(grid):
            entry = Entry(self, width = 4, bd = 2, justify = 'center')
            entry.grid(row = index[0], column = index[1])
            self.entries.append(entry)

    def on_button(self):
        self.values = np.array([ent.get() for ent in self.entries]).reshape(grid.shape)
        self.destroy()

In [319]:
w = SampleApp()

In [320]:
w.mainloop()

In [315]:
w.values

array([['', '', '', '', ''],
       ['', '', '', '', ''],
       ['', '', '', '', ''],
       ['', '', '2', '3', ''],
       ['', '', '', '', '']], dtype='<U1')

In [263]:
import tkinter as tk
import random

class App(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.canvas = tk.Canvas(self, width=500, height=500, borderwidth=0, highlightthickness=0)
        self.canvas.pack(side="top", fill="both", expand="true")
        self.rows = 100
        self.columns = 100
        self.cellwidth = 25
        self.cellheight = 25

        self.rect = {}
        self.oval = {}
        for column in range(20):
            for row in range(20):
                x1 = column*self.cellwidth
                y1 = row * self.cellheight
                x2 = x1 + self.cellwidth
                y2 = y1 + self.cellheight
                self.rect[row,column] = self.canvas.create_rectangle(x1,y1,x2,y2, fill="blue", tags="rect")
                self.oval[row,column] = self.canvas.create_oval(x1+2,y1+2,x2-2,y2-2, fill="blue", tags="oval")

        self.redraw(1000)

    def redraw(self, delay):
        self.canvas.itemconfig("rect", fill="blue")
        self.canvas.itemconfig("oval", fill="blue")
        for i in range(10):
            row = random.randint(0,19)
            col = random.randint(0,19)
            item_id = self.oval[row,col]
            self.canvas.itemconfig(item_id, fill="green")
        self.after(delay, lambda: self.redraw(delay))

In [264]:
app = App()
app.mainloop()